# Importing libraries 

In [9]:
import json
import jsonschema
from   jsonschema import validate
import datetime
import re
import numpy as np
from dateutil.relativedelta import relativedelta
import pandas as pd

# Declaring functions 

#### Validation functions 

In [2]:
def validateJson(jsonData):
    try:
        validate(instance=jsonData, schema=studentSchema)
    except jsonschema.exceptions.ValidationError as err:
        return False
    return True

In [3]:
def validateDate(df, list_of_column_dates):
    for column_name in list_of_column_dates: 
        try:
            pd.to_datetime(df[column_name], format='%Y-%m-%d', errors='raise')
            print('Column '+ str(column_name)+ ' in correct format' )
        except ValueError:
            print('please verify '+ str(column_name)+ ' column')
            pass

In [4]:
## This is an method better than in method 
## https://stackoverflow.com/questions/7571635/fastest-way-to-check-if-a-value-exists-in-a-list

def validateValues(df, column_name, list_of_valid_values):
    df_column = df[column_name]
    c = [0 for x in range(0,len(df_column))]
    reverse_lookup = {x:i for i, x in enumerate(list_of_valid_values)}
    for i, x in enumerate(df_column):
        c[i] = reverse_lookup.get(x, -1)
    
    vls_not_in_pattern = c.count(-1) 
    if (vls_not_in_pattern ==0): 
        print("All elements of "+ str(column_name) +" column respect the pattern")
        return
    else: 
        print("There are "+ str(vls_not_in_pattern)+ " elements that does not follow the pattern in "+str(column_name)+" column.")
        return 

In [5]:
def validateUniqueness(df, columns): 
    for column in columns:
        if (len(df[column]) > df[column].nunique()):
            print("Column "+str(column) +" has not unique values. Please verify.")
        else: 
            print("Column "+str(column) +" has unique values for each register.")

#### Calculation functions

In [6]:
def peso_salario (salario_bruto, cargo, salario_minimo):
    if cargo == "Estagiário":
        return 1 
    if salario_bruto/salario_minimo >8:
        return 1/5
    if salario_bruto/salario_minimo >5:
        return 1/3
    if salario_bruto/salario_minimo >3:
        return 1/2
    else: 
        return 1

In [ ]:
def peso_area(area):
    if area == 'Diretoria': 
        return 1
    if area in ('Contabilidade','Financeiro','Tecnologia'): 
        return 2
    if area == 'Serviços Gerais':
        return 3
    else: 
        return 5

In [ ]:
def peso_tempo_admissao(data_admissao):
    r = relativedelta(pd.to_datetime('now'), datetime.datetime.strptime(data_admissao, '%Y-%m-%d')) 
    if (r.years)>8: 
        return 5
    if (r.years)>3: 
        return 3
    if(r.years)>1:
        return 2
    else: 
        return 1

In [ ]:
def tempo_admissao(data_admissao):
    r = relativedelta(pd.to_datetime('now'), datetime.datetime.strptime(data_admissao, '%Y-%m-%d')) 
    return r.years

In [ ]:
def faixa_salarial (salario_bruto):
    if salario_bruto/salario_minimo >8:
        return 'maior que 8 salários'
    if salario_bruto/salario_minimo >5:
        return 'entre 6 e 8 salários'
    if salario_bruto/salario_minimo >3:
        return 'entre 3 e 5 salários'
    else: 
        return 'até 3 salários'

# Reading data

In [7]:
##If this runs without throuing an exeception, the json is ok. 
%run base_desafio_lucros.json python -m json.tool

# Describe what kind of json you expect.
studentSchema = {
    "matricula": "int",
    "nome": "object",
    "area": "object",
    "cargo": "object",
    "salario_bruto": "object",
    "data_de_admissao": "object",
}

# Opening file 
f = open ('base_desafio_lucros.json', "r", encoding='utf-8')

# Reading from file
data = json.loads(f.read())

# Check if it pass the validate schema
isValid = validateJson(data)
                       
if isValid:
    print("Given JSON data is Valid")
else:
    print("Given JSON data is InValid")

Given JSON data is Valid


# Transforming data 

In [10]:
# Transforming it to table
data_df = pd.json_normalize(data, errors = 'raise')

# Transforming data
data_df['salario_bruto'] = data_df['salario_bruto'].str.replace(r'[^0-9.]', r'', regex=True).astype(float)

# Verifying column format and business rules

Here we suppose that business rules were informed by business people.

In [11]:
list_of_valid_cargos = ['Pessoa Desenvolvedora', 'Jovem Aprendiz',
                        'Auxiliar Administrativo', 'Atendente', 'Líder de Relacionamento',
                        'Copeiro', 'Auxiliar de Ouvidoria', 'Diretor Financeiro',
                        'Contador Júnior', 'Contador Pleno', 'Atendente de Almoxarifado',
                        'Economista Pleno', 'Diretor Tecnologia', 'Economista Júnior',
                        'Líder de Ouvidoria', 'Analista de Finanças', 'Estagiário',
                        'Auxiliar de Contabilidade']

list_of_valid_areas = ['Tecnologia', 'Contabilidade', 'Relacionamento com o Cliente',
                       'Serviços Gerais', 'Diretoria', 'Financeiro']

Could be verifyed in other steps, but we assume that some columns may have problems. 

In [12]:
#Verify date format
validateDate(data_df, ['data_de_admissao'])

#Validate text values (here the main goal is avoiding that business people insert new values without communicating us)
validateValues(data_df, 'cargo', list_of_valid_cargos)
validateValues(data_df, 'area', list_of_valid_areas)

#Validate uniqueness of registers
validateUniqueness(data_df, ['matricula','nome'])

Column data_de_admissao in correct format
All elements of cargo column respect the pattern
All elements of area column respect the pattern
Column matricula has unique values for each register.
Column nome has not unique values. Please verify.


# Calculating scores

In [13]:
salario_minimo = 1212
percent_lucros = 0.2
total_lucros = 5000000
data_df['tempo_admissao'] = data_df["data_de_admissao"].apply(tempo_admissao)
data_df['faixa_salarial'] = data_df["salario_bruto"].apply(faixa_salarial)

data_df['peso_salario'] = data_df[["salario_bruto","cargo"]].apply(lambda x: peso_salario(x["salario_bruto"],x["cargo"],salario_minimo), axis=1)    
data_df['peso_area']  = data_df["area"].apply(peso_area)
data_df['peso_tempo_admissao'] = data_df["data_de_admissao"].apply(peso_tempo_admissao)
data_df['peso_usuario'] = data_df["peso_salario"]*data_df['peso_area']*data_df['peso_tempo_admissao']
data_df['porcao_bonus'] = data_df["peso_usuario"]/data_df["peso_usuario"].sum()
data_df['bonus'] = round(percent_lucros*total_lucros*data_df['porcao_bonus'],2)

# Grouping and presenting manegerial data

In [14]:
## Total distribuído
total_distribuido_bonus = data_df['bonus'].sum()

## Total disponibilizado
total_reservado_bonus = percent_lucros * total_lucros

In [15]:
#Total distribuído e número de funcionários por área
renamed_columns_area  = {'matricula': 'total_de_funcionarios',
                             'bonus': 'total_bonus'}

df_resumo_area = data_df.groupby(["area"])\
                 .agg({"matricula":pd.Series.nunique,
                       "bonus":"sum"})\
                 .reset_index()\
                 .rename(columns=renamed_columns_area)

In [16]:
# Total distribuído e número de funcionários por faixa salarial 
renamed_columns_area  = {'matricula': 'total_de_funcionarios',
                             'bonus': 'total_bonus'}

df_resumo_faixa_salarial = data_df.groupby(["faixa_salarial"])\
                 .agg({"matricula":pd.Series.nunique,
                       "bonus":"sum"})\
                 .reset_index()\
                 .rename(columns=renamed_columns_area)

In [17]:
#Total distribuído e número de funcionários por tempo de admissão
renamed_columns_area  = {'matricula': 'total_de_funcionarios',
                             'bonus': 'total_bonus'}

df_resumo_tempo_admissao = data_df.groupby(["tempo_admissao"])\
                 .agg({"matricula":pd.Series.nunique,
                       "bonus":"sum"})\
                 .reset_index()\
                 .rename(columns=renamed_columns_area)